In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715199477344

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    # tag
    mlflow.set_tag('developer', 'luis')
    
    # data
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    
    # hyperparameter
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    # metrics
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    
    # save model from local file
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [10]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[23:49:22] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.96794                                                    
[1]	validation-rmse:8.41747                                                     
[2]	validation-rmse:7.21531                                                     
[3]	validation-rmse:6.80604                                                     
[4]	validation-rmse:6.65332                                                     
[5]	validation-rmse:6.58603                                                     
[6]	validation-rmse:6.55635                                                     
[7]	validation-rmse:6.54037                                                     
[8]	validation-rmse:6.52829                                                     
[9]	validation-rmse:6.51994                                                     
[10]	validation-rmse:6.51561                                           

[98]	validation-rmse:6.54171                                                    
[99]	validation-rmse:6.54034                                                    
[100]	validation-rmse:6.53913                                                   
[101]	validation-rmse:6.53894                                                   
[102]	validation-rmse:6.53876                                                   
[103]	validation-rmse:6.53751                                                   
[104]	validation-rmse:6.53646                                                   
[105]	validation-rmse:6.53482                                                   
[106]	validation-rmse:6.53431                                                   
[107]	validation-rmse:6.53235                                                   
[108]	validation-rmse:6.53144                                                   
[109]	validation-rmse:6.53128                                                   
[110]	validation-rmse:6.5306

[300]	validation-rmse:6.41993                                                   
[301]	validation-rmse:6.41917                                                   
[302]	validation-rmse:6.41899                                                   
[303]	validation-rmse:6.41875                                                   
[304]	validation-rmse:6.41754                                                   
[305]	validation-rmse:6.41790                                                   
[306]	validation-rmse:6.41722                                                   
[307]	validation-rmse:6.41689                                                   
[308]	validation-rmse:6.41673                                                   
[309]	validation-rmse:6.41647                                                   
[310]	validation-rmse:6.41616                                                   
[311]	validation-rmse:6.41611                                                   
[312]	validation-rmse:6.4161

[502]	validation-rmse:6.36619                                                   
[503]	validation-rmse:6.36590                                                   
[504]	validation-rmse:6.36557                                                   
[505]	validation-rmse:6.36550                                                   
[506]	validation-rmse:6.36518                                                   
[507]	validation-rmse:6.36508                                                   
[508]	validation-rmse:6.36477                                                   
[509]	validation-rmse:6.36452                                                   
[510]	validation-rmse:6.36451                                                   
[511]	validation-rmse:6.36412                                                   
[512]	validation-rmse:6.36401                                                   
[513]	validation-rmse:6.36403                                                   
[514]	validation-rmse:6.3640

[704]	validation-rmse:6.33815                                                   
[705]	validation-rmse:6.33778                                                   
[706]	validation-rmse:6.33761                                                   
[707]	validation-rmse:6.33749                                                   
[708]	validation-rmse:6.33742                                                   
[709]	validation-rmse:6.33742                                                   
[710]	validation-rmse:6.33738                                                   
[711]	validation-rmse:6.33717                                                   
[712]	validation-rmse:6.33706                                                   
[713]	validation-rmse:6.33674                                                   
[714]	validation-rmse:6.33539                                                   
[715]	validation-rmse:6.33553                                                   
[716]	validation-rmse:6.3351

...

[13]	validation-rmse:11.55774                                                   
[14]	validation-rmse:11.17080                                                   
[15]	validation-rmse:10.81132                                                   
[16]	validation-rmse:10.47996                                                   
[17]	validation-rmse:10.17228                                                   
[18]	validation-rmse:9.88768                                                    
[19]	validation-rmse:9.62545                                                    
[20]	validation-rmse:9.38391                                                    
[21]	validation-rmse:9.16072                                                    
[22]	validation-rmse:8.95529                                                    
[23]	validation-rmse:8.76703                                                    
[24]	validation-rmse:8.59276                                                    
[25]	validation-rmse:8.43237

[215]	validation-rmse:6.51172                                                   
[216]	validation-rmse:6.51155                                                   
[217]	validation-rmse:6.51131                                                   
[218]	validation-rmse:6.51119                                                   
[219]	validation-rmse:6.51057                                                   
[220]	validation-rmse:6.51020                                                   
[221]	validation-rmse:6.50983                                                   
[222]	validation-rmse:6.50945                                                   
[223]	validation-rmse:6.50907                                                   
[224]	validation-rmse:6.50869                                                   
[225]	validation-rmse:6.50833                                                   
[226]	validation-rmse:6.50813                                                   
[227]	validation-rmse:6.5078

[417]	validation-rmse:6.46301                                                   
[418]	validation-rmse:6.46278                                                   
[419]	validation-rmse:6.46266                                                   
[420]	validation-rmse:6.46244                                                   
[421]	validation-rmse:6.46232                                                   
[422]	validation-rmse:6.46209                                                   
[423]	validation-rmse:6.46193                                                   
[424]	validation-rmse:6.46181                                                   
[425]	validation-rmse:6.46155                                                   
[426]	validation-rmse:6.46123                                                   
[427]	validation-rmse:6.46111                                                   
[428]	validation-rmse:6.46086                                                   
[429]	validation-rmse:6.4607

[619]	validation-rmse:6.43013                                                   
[620]	validation-rmse:6.42993                                                   
[621]	validation-rmse:6.42974                                                   
[622]	validation-rmse:6.42970                                                   
[623]	validation-rmse:6.42955                                                   
[624]	validation-rmse:6.42947                                                   
[625]	validation-rmse:6.42941                                                   
[626]	validation-rmse:6.42915                                                   
[627]	validation-rmse:6.42903                                                   
[628]	validation-rmse:6.42890                                                   
[629]	validation-rmse:6.42883                                                   
[630]	validation-rmse:6.42869                                                   
[631]	validation-rmse:6.4284

[821]	validation-rmse:6.40434                                                   
[822]	validation-rmse:6.40417                                                   
[823]	validation-rmse:6.40402                                                   
[824]	validation-rmse:6.40392                                                   
[825]	validation-rmse:6.40383                                                   
[826]	validation-rmse:6.40365                                                   
[827]	validation-rmse:6.40361                                                   
[828]	validation-rmse:6.40348                                                   
[829]	validation-rmse:6.40332                                                   
[830]	validation-rmse:6.40320                                                   
[831]	validation-rmse:6.40310                                                   
[832]	validation-rmse:6.40302                                                   
[833]	validation-rmse:6.4028

In [13]:
best_params = {'learning_rate':0.1962868617085126,
          'max_depth':16,
          'min_child_weight':1.0497064443420323,
          'objective':'reg:linear',
          'reg_alpha':0.032197331966541155,
          'reg_lambda':0.22545646859618265,
          'seed':42
         }

mlflow.log_params(best_params)

booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse", rmse)

# save preprocessor locally
with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

# log preprocessor mlflow
mlflow.log_artifact("models/preprocessor.b", artifact_path='preprocessor')

# save model
mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[01:44:57] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.71942
[1]	validation-rmse:15.00108
[2]	validation-rmse:12.89387
[3]	validation-rmse:11.28363
[4]	validation-rmse:10.06613
[5]	validation-rmse:9.15416
[6]	validation-rmse:8.47936
[7]	validation-rmse:7.98223
[8]	validation-rmse:7.61900
[9]	validation-rmse:7.35472
[10]	validation-rmse:7.16139
[11]	validation-rmse:7.01690
[12]	validation-rmse:6.91023
[13]	validation-rmse:6.83021
[14]	validation-rmse:6.76992
[15]	validation-rmse:6.72468
[16]	validation-rmse:6.68876
[17]	validation-rmse:6.66141
[18]	validation-rmse:6.63987
[19]	validation-rmse:6.62236
[20]	validation-rmse:6.60872
[21]	validation-rmse:6.59855
[22]	validation-rmse:6.58982
[23]	validation-rmse:6.58268
[24]	validation-rmse:6.57843
[25]	validation-rmse:6.57437
[26]	validation-rmse:6.57007
[27]	validation-rmse:6.56655
[28]	validation-rmse:6.56355
[29]	validation-rmse:6.56021
[30]	validation-

[273]	validation-rmse:6.39003
[274]	validation-rmse:6.38952
[275]	validation-rmse:6.38966
[276]	validation-rmse:6.38903
[277]	validation-rmse:6.38875
[278]	validation-rmse:6.38855
[279]	validation-rmse:6.38808
[280]	validation-rmse:6.38774
[281]	validation-rmse:6.38755
[282]	validation-rmse:6.38741
[283]	validation-rmse:6.38734
[284]	validation-rmse:6.38708
[285]	validation-rmse:6.38666
[286]	validation-rmse:6.38665
[287]	validation-rmse:6.38620
[288]	validation-rmse:6.38585
[289]	validation-rmse:6.38579
[290]	validation-rmse:6.38545
[291]	validation-rmse:6.38532
[292]	validation-rmse:6.38488
[293]	validation-rmse:6.38423
[294]	validation-rmse:6.38370
[295]	validation-rmse:6.38351
[296]	validation-rmse:6.38305
[297]	validation-rmse:6.38268
[298]	validation-rmse:6.38244
[299]	validation-rmse:6.38228
[300]	validation-rmse:6.38223
[301]	validation-rmse:6.38176
[302]	validation-rmse:6.38139
[303]	validation-rmse:6.38124
[304]	validation-rmse:6.38090
[305]	validation-rmse:6.38039
[306]	vali

[547]	validation-rmse:6.32857
[548]	validation-rmse:6.32833
[549]	validation-rmse:6.32779
[550]	validation-rmse:6.32766
[551]	validation-rmse:6.32764
[552]	validation-rmse:6.32734
[553]	validation-rmse:6.32726
[554]	validation-rmse:6.32701
[555]	validation-rmse:6.32683
[556]	validation-rmse:6.32659
[557]	validation-rmse:6.32647
[558]	validation-rmse:6.32637
[559]	validation-rmse:6.32639
[560]	validation-rmse:6.32628
[561]	validation-rmse:6.32609
[562]	validation-rmse:6.32593
[563]	validation-rmse:6.32606
[564]	validation-rmse:6.32585
[565]	validation-rmse:6.32536
[566]	validation-rmse:6.32541
[567]	validation-rmse:6.32557
[568]	validation-rmse:6.32558
[569]	validation-rmse:6.32549
[570]	validation-rmse:6.32526
[571]	validation-rmse:6.32482
[572]	validation-rmse:6.32458
[573]	validation-rmse:6.32459
[574]	validation-rmse:6.32486
[575]	validation-rmse:6.32485
[576]	validation-rmse:6.32471
[577]	validation-rmse:6.32467
[578]	validation-rmse:6.32450
[579]	validation-rmse:6.32457
[580]	vali

[821]	validation-rmse:6.29949
[822]	validation-rmse:6.29943
[823]	validation-rmse:6.29922
[824]	validation-rmse:6.29918
[825]	validation-rmse:6.29916
[826]	validation-rmse:6.29924
[827]	validation-rmse:6.29931
[828]	validation-rmse:6.29925
[829]	validation-rmse:6.29914
[830]	validation-rmse:6.29901
[831]	validation-rmse:6.29888
[832]	validation-rmse:6.29860
[833]	validation-rmse:6.29850
[834]	validation-rmse:6.29847
[835]	validation-rmse:6.29834
[836]	validation-rmse:6.29820
[837]	validation-rmse:6.29829
[838]	validation-rmse:6.29819
[839]	validation-rmse:6.29822
[840]	validation-rmse:6.29815
[841]	validation-rmse:6.29810
[842]	validation-rmse:6.29799
[843]	validation-rmse:6.29802
[844]	validation-rmse:6.29788
[845]	validation-rmse:6.29779
[846]	validation-rmse:6.29771
[847]	validation-rmse:6.29768
[848]	validation-rmse:6.29765
[849]	validation-rmse:6.29770
[850]	validation-rmse:6.29756
[851]	validation-rmse:6.29747
[852]	validation-rmse:6.29751
[853]	validation-rmse:6.29748
[854]	vali

ModelInfo(artifact_path='models_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.9.12', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/2af7f410858a4425b7290525fdf8071a/models_mlflow', model_uuid='9e48b8bea7fb46288eeee67050c76ada', run_id='2af7f410858a4425b7290525fdf8071a', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-06-16 01:45:56.711189', mlflow_version='1.26.1')

In [14]:
logged_model = 'runs:/2af7f410858a4425b7290525fdf8071a/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

[01:54:04] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 2af7f410858a4425b7290525fdf8071a

In [15]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[01:55:21] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

# Autolog parameters
mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)